# Install


In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
import os

import pandas as pd
from dotenv import load_dotenv

import utils
import mock_data
import model
import data_import

load_dotenv("../user.env")

True

# Parse


In [2]:
import copy


def parse_current_account_details(details):
    adjusted_details = copy.deepcopy(details)
    adjusted_details = utils.rename_keys(
        adjusted_details,
        {
            "resourceId": utils.to_snake_case_with_dots("resourceId"),
            "bban": utils.to_snake_case_with_dots("bban"),
            "currency": utils.to_snake_case_with_dots("currency"),
            "name": utils.to_snake_case_with_dots("name"),
            "cashAccountType": utils.to_snake_case_with_dots("cashAccountType"),
        },
    )
    adjusted_details = model.CurrentAccountDetails.model_validate(adjusted_details)
    return adjusted_details


def parse_card_details(details):
    adjusted_details = copy.deepcopy(details)
    adjusted_details = utils.rename_keys(
        adjusted_details,
        {
            "resourceId": utils.to_snake_case_with_dots("resourceId"),
            "currency": utils.to_snake_case_with_dots("currency"),
            "cashAccountType": utils.to_snake_case_with_dots("cashAccountType"),
            "maskedPan": utils.to_snake_case_with_dots("maskedPan"),
            "details": utils.to_snake_case_with_dots("details"),
        },
    )
    credit_card_details = model.CreditCardDetails.model_validate(adjusted_details)
    return credit_card_details


def parse_account_details(account_data):
    details = account_data.get("account")
    if details.get("cashAccountType") == "CACC":
        account_details = parse_current_account_details(details)
        return account_details

    if details.get("cashAccountType") == "CARD":
        card_details = parse_card_details(details)
        return card_details


def parse_metadata(data):
    account_metadata = model.AccountMetadata.model_validate(data.get("metadata"))
    return account_metadata


def parse_account_balances(account_data):
    def get_balances(balance_data):
        return [model.Balance.model_validate(balance) for balance in balance_data]

    balance_data = copy.deepcopy(account_data.get("balances"))
    balance_data = utils.rename_keys(
        balance_data,
        {
            "balanceAmount": utils.to_snake_case_with_dots("balanceAmount"),
            "balanceType": utils.to_snake_case_with_dots("balanceType"),
            "referenceDate": utils.to_snake_case_with_dots("referenceDate"),
        },
    )
    return model.AccountBalances(balances=get_balances(balance_data))

In [3]:
def load_user_data():
    # user level
    data, requisition = mock_data.load_data()
    metadata = model.UserAccountMetadata(accounts={})
    details = model.UserAccountDetails()
    balances = model.UserAccountBalances()
    transactions = {}
    # import an account
    for account_id in requisition.accounts:

        # read all data
        account_data = data.get(account_id)

        # parse and validate
        metadata.add_account(
            account_id=account_id, account_metadata=parse_metadata(account_data)
        )
        details.add_account(
            account_id=account_id, account=parse_account_details(account_data)
        )
        balances.set_account_balances(
            account_id=account_id, balance=parse_account_balances(account_data)

        )

        transactions[account_id] = account_data.get('transactions')

    user_data = model.UserData(
        requisition=requisition,
        account_metadata=metadata,
        account_details=details,
        account_balances=balances)
    return user_data, transactions

user_data,transactions = load_user_data()

In [13]:
def parse_account_transactions(transactions, account_id):
    booked = pd.json_normalize(transactions.get(account_id).get("booked"))
    pending = pd.json_normalize(transactions.get(account_id).get("pending"))
    if len(booked) != 0:
        booked["status"] = "booked"

    if len(pending) != 0:
        pending["status"] = "pending"
    df = pd.concat([booked, pending])
    account_transactions = df.to_dict(orient="records")
    account_transactions = utils.clean_column_names(account_transactions)
    account_transactions = [model.Transaction(**element) for element in  account_transactions]
    return account_transactions

In [15]:
account_id = user_data.requisition.accounts[0]
account_data = 
account_transactions = parse_account_transactions(transactions=transactions, account_id=account_id)

In [16]:
# add transactions
user_data.account_details.fetch_account_details_by_id

UserAccountDetails(credit_cards={'d2ff77d0-6c80-4580-95a5-e3e87a098db9': CreditCardDetails(resource_id='c4e9a964-b676-43c1-81f9-5479f8894604', currency='GBP', masked_pan='**4572', details='STAFF ALL IN ONE CREDIT CARD', account_id='d2ff77d0-6c80-4580-95a5-e3e87a098db9')}, current_accounts={'590300bd-3daf-4d5e-9274-7a3782261f7e': CurrentAccountDetails(resource_id='cbc85e1c-cdbc-4e3a-9918-03b74d79e719', bban='09012921263671', currency='GBP', name='joint account', cash_account_type='CACC', account_id='590300bd-3daf-4d5e-9274-7a3782261f7e'), 'e9e5f8b9-da61-49ce-bdae-56546ce4a1c9': CurrentAccountDetails(resource_id='a2b51663-5cf3-43d6-866e-65e5c63542bc', bban='09012916688559', currency='GBP', name='single account', cash_account_type='CACC', account_id='e9e5f8b9-da61-49ce-bdae-56546ce4a1c9')})

# Process data from API (mocked)


In [ ]:
def extract_transactions_and_balance_from_response(account_id: str, transactions_raw):
    def combine_pending_and_booked_transactions(transactions: list[dict]):
        booked_df = pd.json_normalize(transactions.get("booked"))
        if booked_df is not None:
            booked_df["status"] = "booked"
        pending_df = pd.json_normalize(transactions.get("pending"))
        if pending_df is not None:
            pending_df["status"] = "pending"

        df = pd.concat([booked_df, pending_df])
        transactions = df.to_dict(orient="records")
        return transactions

    def find_account_transactions():
        # loop through looking for account id
        result = None
        finished = False

        for acc in transactions_raw:
            acc_id = acc[0]
            if acc_id == account_id:
                result = acc
        finished = True

        if finished and result is None:
            Exception(f"could not find account_id {account_id} in account details data")

        return result

    def extract_transactions(result):
        acc_transactions = result[1][0].get("transactions")
        return acc_transactions

    def extract_balances(result):
        acc_balances = result[1][1].get("balances")
        return acc_balances

    def add_account_id_to_transactions(transactions: list[dict], account_id: str):
        result = []

        for transaction in transactions:
            updated_dict = transaction
            updated_dict["account_id"] = account_id
            result.append(updated_dict)

        return result

    # parse
    result = find_account_transactions()
    acc_transactions = extract_transactions(result)
    acc_balances = extract_balances(result)

    # post process
    acc_transactions = combine_pending_and_booked_transactions(acc_transactions)
    acc_transactions = add_account_id_to_transactions(
        transactions=acc_transactions, account_id=account_id
    )

    return acc_transactions, acc_balances


def populate_user_accounts_data(
    requisition: model.Requisition,
    transactions,
    account_details: model.UserAccountDetails,
):
    # Populate accounts
    accounts = [
        parse_account_dict(acc, id=requisition.accounts[idx])
        for idx, acc in enumerate(account_details)
    ]

    for account_id in requisition.accounts:
        # acc_position = requisition.get("accounts").index(account_id)
        acc_position = requisition.accounts.index(account_id)
        account_name = accounts[acc_position].get("account_name")
        print(
            f"account_position={acc_position}, account_name={account_name}, account_id={account_id}"
        )

        transactions, balances = extract_transactions_and_balance_from_response(
            account_id, transactions_raw=transactions_raw
        )

        accounts[acc_position]["transactions"] = transactions
        accounts[acc_position]["balances"] = balances

    return accounts

In [ ]:
user_data, transactions, account_mapping = mock_data.get_mocked_data()

In [ ]:
data_import.set_account_ids(user_data=user_data, mapping=account_mapping)

In [ ]:
placement_index = 0
account_id = transactions[placement_index][0]

In [ ]:
user_data.account_balances

In [ ]:
for account_id in requisition.accounts:
    # acc_position = requisition.get("accounts").index(account_id)
    acc_position = requisition.accounts.index(account_id)
    account_name = accounts[acc_position].get("account_name")
    print(
        f"account_position={acc_position}, account_name={account_name}, account_id={account_id}"
    )

    transactions, balances = extract_transactions_and_balance_from_response(
        account_id, transactions_raw=transactions_raw
    )

    accounts[acc_position]["transactions"] = transactions
    accounts[acc_position]["balances"] = balances

In [ ]:
user_accounts_data = populate_user_accounts_data(
    requisition=user_data.requisition,
    transactions=transactions,
    account_details=user_data.account_details,
)

In [ ]:
type(user_accounts_data)

# Parsed data to domain model

accounts


In [ ]:
def get_account_data(account_data):
    result_dict = {}
    balances: list[dict] = account_data.get("balances")
    data_list = utils.flatten_and_remove_duplicates_from_dictionary(balances)
    for entry in data_list:
        balance_type = entry.get("balanceType")
        amount = entry.get("amount")

        if balance_type and amount:
            key = f"{balance_type}Balance"
            result_dict[key] = amount

    result_dict["currency"] = data_list[0]["currency"]
    result_dict["referenceDate"] = data_list[0]["referenceDate"]
    result_dict = utils.clean_column_names(result_dict)

    acc_detail = {
        "account_id": account_data.get("account_id"),
        "account_name": account_data.get("account_name"),
        "account_type": account_data.get("account_type"),
        "currency": account_data.get("currency"),
    }
    acc_detail.update(result_dict)

    return acc_detail


# clean
def transactions_dict_to_model(account_data: dict, account_id: str):
    transactions = utils.clean_column_names(account_data.get("transactions"))
    transactions = [model.Transaction.from_dict(d) for d in transactions]
    filtered_transactions = [t for t in transactions if t.account_id == account_id]
    return transactions

In [ ]:
def get_user_selection(selected_account: int = 0):
    account_ids = [d.get("account_id") for d in user_accounts_data]
    if selected_account >= len(account_ids):
        raise ValueError("invalid entry")
    account_id = account_ids[selected_account]
    account_position = account_ids.index(account_id)
    return account_position


account_position = get_user_selection()

In [ ]:
# user selects account
user_accounts = []
for i in range(0, len(user_accounts_data)):
    try:
        account_data = user_accounts_data[i]
        account_type = account_data.get("account_type")
        # initialise account
        if account_type == "CACC":
            account = model.BankAccount.from_dict(get_account_data(account_data))
        elif account_type == "CARD":
            account = model.CreditCard.from_dict(get_account_data(account_data))

        # initialise transactions
        transactions = transactions_dict_to_model(account_data, account.account_id)
        account.add_transactions(transactions=transactions)
        user_accounts.append(account)
    except Exception as e:
        print(f"failed to process account {i}")
        raise e

In [ ]:
user_accounts

In [ ]:
user = model.User(user_name="Matthew")

In [ ]:
user = model.User(user_name="Matthew")
_ = [user.add_account(account) for account in user_accounts]

In [ ]:
transactions = user.get_transactions()
transactions_by_amount = user.get_transactions_by_amount(100, 200)
transactions_by_amount = user.get_transactions_by_amount(400, 500)

In [ ]:
from collections import OrderedDict


def find_common_transactions():
    ...


def find_common_dicts(*lists):
    def dict_to_ordered_tuple(d):
        return tuple(OrderedDict(sorted(d.items())).items())

    sets = [set(dict_to_ordered_tuple(d) for d in lst) for lst in lists]

    common_set = set.intersection(*sets)

    result = [dict(t) for t in common_set]
    return result

In [ ]:
queries = [user.get_transactions(), user.get_transactions_by_amount(-100, -10)]
dicts = [[d.to_dict() for d in query] for query in queries]
common_dicts = find_common_dicts(*dicts)

In [ ]:
query_response = [model.Transaction(**t) for t in common_dicts]

In [ ]:
def transactions_to_dataframe(transactions: list[model.Transaction]):
    return pd.DataFrame(query_response).dropna(axis=1)

In [ ]:
transactions_to_dataframe(query_response)